In [1]:
import transformers
print(transformers.__version__)

4.48.3


In [9]:
import pandas as pd
from transformers import LongT5ForConditionalGeneration, T5Tokenizer

model_name = "google/long-t5-tglobal-base"

tokenizer = T5Tokenizer.from_pretrained(model_name)
model = LongT5ForConditionalGeneration.from_pretrained(model_name)



def summarize_with_longt5(text, max_length=500):
    inputs = tokenizer(text, return_tensors="pt", max_length=4096, truncation=True)
    summary_ids = model.generate(inputs.input_ids, max_length=max_length, min_length=30, length_penalty=2.0, num_beams=4)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)



# Load Sapegno comments
# sapegno = pd.read_csv("/home/rfflpllcn/IdeaProjects/divine_semantics_db/divine_semantics_db/data/comments/sapegno/inferno/sapegno_inferno_comment.csv")
sapegno = pd.read_csv("/content/sapegno_inferno_comment.csv")
sapegno = sapegno.iloc[:3]


sapegno["summary"] = sapegno["text"].apply(lambda x: summarize_with_longt5(str(x)) if pd.notna(x) else "")

# Save summarized version
sapegno[["cantica", "canto", "start_verse", "end_verse", "author", "summary"]].to_csv("sapegno_inferno_summarized.csv", index=False)
